In [ ]:
import random
import time
import copy
from collections import deque

import numpy as np

import torch as tc
import torch.optim as opt

from model import SnakeNet
from core import CUDA_AVAILABLE, DEVICE
from config import NROW,NCOL,EPISODE_MAXLEN
from per import PER
from env import Env

#Train Parameters

BATCH_SIZE = 32
DISC_RATIO=0.9
EPISODE_CNT=10000
EPS_EXPONENT=2.7
EPS_MAX=1.0
EPS_MIN=0.01
LEARN_FREQ=5
LEARNING_RATE=1e-3
REPLAY_MEM_SIZE = 200000
STAT_DISPLAY_FREQ = 10
SAVE_TEMP_FREQ = 100
TARGET_UPD_FREQ = LEARN_FREQ*500

import time as timelib
from PIL import Image
from PIL import ImageDraw
from torchvision import transforms
import matplotlib.pyplot as plt
from IPython import display
def render(state,sleep_time=0,clear=True):
    #print(obs)
    txt = Image.new("RGBA", (NCOL*30,NROW*30), (10,50,100,100))
    draw = ImageDraw.Draw(txt)
    grid_size = 30
    for i in range(NROW):
        for j in range(NCOL):
            y1 = i*grid_size
            x1 = j*grid_size
            y2 = (i+1)*grid_size
            x2 = (j+1)*grid_size
            draw.rectangle(((x1, y1), (x2, y2)), outline='black', width=1)
    for y,x in state['foods']:
        draw.ellipse((x*grid_size, y*grid_size, (x+1)*grid_size, (y+1)*grid_size), fill = 'yellow', outline ='yellow')
    for y,x in state['snake']:
        if (y,x)==state['snake'][-1]:
            draw.polygon([((x+1/2)*grid_size,y*grid_size),(x*grid_size,(y+1/2)*grid_size),
                ((x+1/2)*grid_size,(y+1)*grid_size), ((x+1)*grid_size,(y+1/2)*grid_size)], fill = 'red')
        else:
            draw.rectangle(((x*grid_size, y*grid_size), ((x+1)*grid_size, (y+1)*grid_size)), fill='red', outline='red')
    txt = txt.resize((64, 64*NROW//NCOL)).convert('RGB')
    numpy_image = np.array(txt)
    plt.axis("off")
    trans1 = transforms.ToTensor()
    tensor_image = trans1(numpy_image)
    tf = transforms.ToPILImage()
    plt.imshow(tf(tensor_image))
    plt.show()
    plt.close()
    if sleep_time:
        timelib.sleep(sleep_time)
    if clear:
        display.clear_output(wait=True)
    return tensor_image

def state2input(state):
    #5 = shape, head pos, tailpos, Food Pos, Step
    ret=[[[0]*NCOL for _ in range(NROW)] for __ in range(4)]
    snake=state['snake']
    for i,(y,x) in enumerate(snake):
        ret[0][y][x]=(i+1)/len(snake)
    heady,headx = snake[-1]
    ret[1][heady][headx]=1.
    taily,tailx = snake[0]
    ret[2][taily][tailx]=1.
    for y,x in state['foods']:
        ret[3][y][x]=1.
    return ret

print("CUDA: ",CUDA_AVAILABLE)
net=SnakeNet().cuda() if CUDA_AVAILABLE else SnakeNet()
net.load_state_dict(tc.load('./netw.pt'))
net.eval()

with tc.no_grad():
    env = Env()
    state=env.reset()
    rwdsum=0
    while not state['done']:
        res = net(tc.tensor(state2input(state)).to(DEVICE)).detach().squeeze()
        state,rwd = env.step(tc.argmax(res))
        rwdsum+=rwd
        print(rwdsum)
        #print(state2input(state))
        render(state,0.1)